In [24]:
# Importações específicas para EfficientNet e experimentação científica
import os
import numpy as np
import pandas as pd
import cv2
import random
import time
import psutil
from datetime import timedelta, datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
import tensorflow as tf
from keras.applications import EfficientNetB0
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from collections import Counter
from pathlib import Path
import pickle

# Configuração de reprodutibilidade
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Configurações de GPU para EfficientNet (mais otimizações)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU configurada para crescimento dinâmico: {len(gpus)} dispositivos")
    except RuntimeError as e:
        print(f"Erro na configuração GPU: {e}")

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponível: {tf.config.list_physical_devices('GPU')}")
print("EfficientNet configurado para experimentação científica")

GPU configurada para crescimento dinâmico: 1 dispositivos
TensorFlow version: 2.20.0
GPU disponível: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
EfficientNet configurado para experimentação científica


In [25]:
# Configurações otimizadas para EfficientNet
IMG_SIZE = 224  # EfficientNet-B0 usa 224x224 como padrão
BATCH_SIZE = 32  # Mantém consistência para comparação
EPOCHS = 100
VALIDATION_SPLIT = 0.3

# Configurações específicas do EfficientNet
EFFICIENTNET_CONFIG = {
    'base_learning_rate': 0.001,      # Taxa inicial mais alta
    'fine_tune_learning_rate': 0.0001, # Taxa para fine-tuning
    'dropout_rate': 0.5,              # Dropout principal
    'fine_tune_layers': 20,           # Últimas camadas para fine-tuning
    'normalization': 'efficientnet'   # Normalização específica (-1 a 1)
}

# Mapeamento das 7 emoções básicas (igual ao ResNet50)
EMOTION_LABELS = {
    'anger': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 
    'neutral': 4, 'sadness': 5, 'surprise': 6
}

print("Configurações EfficientNet definidas:")
print(f"- Arquitetura: EfficientNet-B0")
print(f"- Tamanho da imagem: {IMG_SIZE}x{IMG_SIZE}")
print(f"- Batch size: {BATCH_SIZE}")
print(f"- Learning rate base: {EFFICIENTNET_CONFIG['base_learning_rate']}")
print(f"- Learning rate fine-tuning: {EFFICIENTNET_CONFIG['fine_tune_learning_rate']}")
print(f"- Normalização: {EFFICIENTNET_CONFIG['normalization']}")
print(f"- Classes de emoção: {len(EMOTION_LABELS)}")
print("- Dados já pré-processados e balanceados")


Configurações EfficientNet definidas:
- Arquitetura: EfficientNet-B0
- Tamanho da imagem: 224x224
- Batch size: 32
- Learning rate base: 0.001
- Learning rate fine-tuning: 0.0001
- Normalização: efficientnet
- Classes de emoção: 7
- Dados já pré-processados e balanceados


In [26]:
class EfficientNetMonitor:
    """
    Classe especializada para monitorar EfficientNet durante treinamento.
    Inclui métricas específicas de eficiência computacional.
    """
    
    def __init__(self):
        self.start_time = None
        self.end_time = None
        self.peak_memory_mb = 0
        self.initial_memory_mb = 0
        self.phase1_time = 0
        self.phase2_time = 0
        self.process = psutil.Process()
        self.training_phases = {'phase1': None, 'phase2': None}
        
    def start_monitoring(self):
        """Inicia o monitoramento com foco em eficiência do EfficientNet"""
        self.start_time = time.time()
        self.initial_memory_mb = self._get_memory_usage()
        self.peak_memory_mb = self.initial_memory_mb
        print(f"Iniciando treinamento EfficientNet-B0...")
        print(f"Horário de início: {time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Memória inicial: {self.initial_memory_mb:.2f} MB")
        print(f"Objetivo: Máxima eficiência computacional")
        print("-" * 50)
        
    def start_phase(self, phase_name):
        """Inicia uma fase específica do treinamento"""
        self.training_phases[phase_name] = time.time()
        print(f"Iniciando {phase_name} - EfficientNet")
        
    def end_phase(self, phase_name):
        """Finaliza uma fase específica do treinamento"""
        if self.training_phases[phase_name] is not None:
            phase_duration = time.time() - self.training_phases[phase_name]
            if phase_name == 'phase1':
                self.phase1_time = phase_duration
            elif phase_name == 'phase2':
                self.phase2_time = phase_duration
            print(f"{phase_name} concluída em: {timedelta(seconds=int(phase_duration))}")
            return phase_duration
        return 0
        
    def _get_memory_usage(self):
        """Retorna uso atual de memória em MB"""
        return self.process.memory_info().rss / 1024 / 1024
        
    def update_peak_memory(self):
        """Atualiza o pico de memória se necessário"""
        current_memory = self._get_memory_usage()
        if current_memory > self.peak_memory_mb:
            self.peak_memory_mb = current_memory
            
    def get_efficiency_metrics(self):
        """Calcula métricas de eficiência específicas do EfficientNet"""
        current_memory = self._get_memory_usage()
        total_time = self.phase1_time + self.phase2_time
        
        return {
            'memory_efficiency': self.initial_memory_mb / self.peak_memory_mb if self.peak_memory_mb > 0 else 0,
            'time_efficiency': total_time / 3600,  # Horas
            'peak_memory_gb': self.peak_memory_mb / 1024,
            'memory_growth_factor': self.peak_memory_mb / self.initial_memory_mb if self.initial_memory_mb > 0 else 1,
            'phase1_ratio': self.phase1_time / total_time if total_time > 0 else 0,
            'phase2_ratio': self.phase2_time / total_time if total_time > 0 else 0
        }
        
    def end_monitoring(self):
        """Finaliza o monitoramento e exibe estatísticas detalhadas"""
        self.end_time = time.time()
        
        # Calcula tempo total
        total_time_seconds = self.end_time - self.start_time
        total_time_formatted = str(timedelta(seconds=int(total_time_seconds)))
        
        # Memória final
        final_memory_mb = self._get_memory_usage()
        memory_increase = final_memory_mb - self.initial_memory_mb
        
        # Métricas de eficiência
        efficiency_metrics = self.get_efficiency_metrics()
        
        print("\n" + "="*70)
        print("RELATÓRIO DE MONITORAMENTO - EFFICIENTNET-B0")
        print("="*70)
        print(f"Tempo total de treinamento: {total_time_formatted}")
        print(f"  • Fase 1 (Base layers): {timedelta(seconds=int(self.phase1_time))}")
        print(f"  • Fase 2 (Fine-tuning): {timedelta(seconds=int(self.phase2_time))}")
        print(f"Memória inicial: {self.initial_memory_mb:.2f} MB")
        print(f"Memória final: {final_memory_mb:.2f} MB")
        print(f"Pico de memória: {self.peak_memory_mb:.2f} MB")
        print(f"Eficiência de memória: {efficiency_metrics['memory_efficiency']:.3f}")
        print(f"Fator de crescimento: {efficiency_metrics['memory_growth_factor']:.2f}x")
        print("="*70)
        
        return {
            'total_time_seconds': total_time_seconds,
            'total_time_formatted': total_time_formatted,
            'initial_memory_mb': self.initial_memory_mb,
            'final_memory_mb': final_memory_mb,
            'peak_memory_mb': self.peak_memory_mb,
            'memory_increase_mb': memory_increase,
            'phase1_time': self.phase1_time,
            'phase2_time': self.phase2_time,
            'efficiency_metrics': efficiency_metrics
        }

# Instancia o monitor específico para EfficientNet
monitor = EfficientNetMonitor()
print("Monitor EfficientNet inicializado")
print("Recursos de monitoramento: Tempo por fase, Eficiência de memória, Métricas comparativas")

Monitor EfficientNet inicializado
Recursos de monitoramento: Tempo por fase, Eficiência de memória, Métricas comparativas


# MUDAR AQUI TBM DEPOIS QUE A LU MANDAR 

In [21]:
def load_preprocessed_data_efficientnet_from_images():
    """
    Carrega dados pré-processados de imagens JPG com normalização específica para EfficientNet.
    VERSÃO CORRIGIDA para caminhos Windows e emoções em português.
    """
    import cv2
    import os
    import numpy as np
    from collections import Counter
    
    print("Carregando dados pré-processados JPG para EfficientNet...")
    
    # Configurações
    IMG_SIZE = 224  # Tamanho para EfficientNet
    BASE_PATH = r"../data/augmented/raf_db_balanced"  # Seu caminho
    
    # CORRIGIDO: Mapeamento das emoções em português
    EMOTION_LABELS = {
        'Raiva': 0, 'Nojo': 1, 'Medo': 2, 'Felicidade': 3, 
        'Neutro': 4, 'Tristeza': 5, 'Surpresa': 6
    }
    
    def load_images_from_directory(directory_path, set_name):
        """Carrega imagens de um diretório usando os.path.join ao invés de /"""
        images = []
        labels = []
        
        print(f"Carregando {set_name} de: {directory_path}")
        
        # Verifica se o diretório existe
        if not os.path.exists(directory_path):
            print(f"❌ Diretório não encontrado: {directory_path}")
            return np.array([]), np.array([])
        
        # Lista subdiretórios (emoções)
        subdirs = [d for d in os.listdir(directory_path) 
                  if os.path.isdir(os.path.join(directory_path, d))]
        
        print(f"📁 Subdiretórios encontrados: {subdirs}")
        
        for emotion, label in EMOTION_LABELS.items():
            # CORRIGIDO: Usar os.path.join ao invés de /
            emotion_path = os.path.join(directory_path, emotion)
            
            if not os.path.exists(emotion_path):
                print(f"⚠️  Pasta '{emotion}' não encontrada em {directory_path}")
                print(f"    Tentando variações de nome...")
                
                # Tenta variações do nome da emoção
                emotion_variations = [
                    emotion.lower(),
                    emotion.upper(), 
                    emotion.capitalize(),
                    emotion.replace('ç', 'c'),  # Felicidade -> Felicidade
                    emotion.replace('ã', 'a')   # Raiva -> Raiva
                ]
                
                found = False
                for variation in emotion_variations:
                    test_path = os.path.join(directory_path, variation)
                    if os.path.exists(test_path):
                        emotion_path = test_path
                        print(f"    ✅ Encontrado: {variation}")
                        found = True
                        break
                
                if not found:
                    print(f"    ❌ Nenhuma variação encontrada para '{emotion}'")
                    continue
            
            # Carrega imagens da pasta da emoção
            count = 0
            image_files = []
            
            # Busca diferentes extensões
            for ext in ['*.jpg', '*.jpeg', '*.png', '*.bmp']:
                import glob
                pattern = os.path.join(emotion_path, ext)
                image_files.extend(glob.glob(pattern))
            
            print(f"  📸 {emotion}: {len(image_files)} arquivos encontrados")
            
            for img_file in image_files:
                try:
                    # Carrega imagem
                    img = cv2.imread(img_file)
                    if img is None:
                        print(f"    ⚠️ Não foi possível carregar: {os.path.basename(img_file)}")
                        continue
                    
                    # Converte BGR para RGB
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    
                    # Redimensiona se necessário
                    if img.shape[:2] != (IMG_SIZE, IMG_SIZE):
                        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
                    
                    # Garante que seja RGB (3 canais)
                    if len(img.shape) == 2:
                        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                    elif img.shape[2] == 1:
                        img = np.repeat(img, 3, axis=2)
                    elif img.shape[2] == 4:  # RGBA
                        img = img[:, :, :3]  # Remove canal alpha
                    
                    images.append(img)
                    labels.append(label)
                    count += 1
                    
                except Exception as e:
                    print(f"    ❌ Erro ao carregar {os.path.basename(img_file)}: {e}")
                    continue
            
            print(f"  ✅ {emotion}: {count} imagens carregadas com sucesso")
        
        return np.array(images), np.array(labels)
    
    def detect_data_structure(base_path):
        """Detecta a estrutura dos dados automaticamente"""
        print(f"🔍 Analisando estrutura de: {base_path}")
        
        if not os.path.exists(base_path):
            print(f"❌ Caminho base não existe: {base_path}")
            return None
            
        # Lista conteúdo do diretório
        contents = os.listdir(base_path)
        dirs = [d for d in contents if os.path.isdir(os.path.join(base_path, d))]
        files = [f for f in contents if os.path.isfile(os.path.join(base_path, f))]
        
        print(f"📁 Diretórios: {dirs}")
        print(f"📄 Arquivos: {len(files)} encontrados")
        
        # Verifica se tem estrutura train/test
        if 'train' in dirs and 'test' in dirs:
            print("✅ Estrutura detectada: train/test/emotion/")
            return 'train_test'
        
        # Verifica se as pastas são emoções diretamente
        emotion_names = set(EMOTION_LABELS.keys())
        found_emotions = set(dirs) & emotion_names
        
        if found_emotions:
            print(f"✅ Estrutura detectada: emotion/ direta - Emoções: {found_emotions}")
            return 'emotion_direct'
        
        # Verifica variações de nomes
        emotion_variations = []
        for emotion in EMOTION_LABELS.keys():
            variations = [emotion.lower(), emotion.upper(), emotion.capitalize()]
            emotion_variations.extend(variations)
        
        found_variations = set(dirs) & set(emotion_variations)
        if found_variations:
            print(f"✅ Estrutura detectada: emotion/ com variações - Encontradas: {found_variations}")
            return 'emotion_direct'
        
        print("⚠️ Estrutura não reconhecida automaticamente")
        return 'unknown'
    
    try:
        # Detecta estrutura automaticamente
        structure = detect_data_structure(BASE_PATH)
        
        if structure == 'train_test':
            # Estrutura: base/train/emotion/ e base/test/emotion/
            train_path = os.path.join(BASE_PATH, "train")
            test_path = os.path.join(BASE_PATH, "test")
            
            X_train, y_train = load_images_from_directory(train_path, "TREINO")
            X_test, y_test = load_images_from_directory(test_path, "TESTE")
            
        elif structure == 'emotion_direct':
            # Estrutura: base/emotion/ - precisa criar train/test split
            print("📊 Carregando todas as imagens e criando divisão train/test...")
            
            all_images, all_labels = load_images_from_directory(BASE_PATH, "TODAS AS IMAGENS")
            
            if len(all_images) == 0:
                print("❌ Nenhuma imagem carregada!")
                return None, None, None, None
            
            # Cria divisão train/test
            from sklearn.model_selection import train_test_split
            
            X_train, X_test, y_train, y_test = train_test_split(
                all_images, all_labels,
                test_size=0.2,
                stratify=all_labels,
                random_state=42
            )
            
            print("✅ Divisão train/test criada automaticamente (80/20)")
            
        else:
            print("❌ Estrutura de dados não suportada!")
            print("💡 Estruturas esperadas:")
            print("   1. base/train/Raiva/*.jpg, base/train/Nojo/*.jpg, etc.")
            print("   2. base/Raiva/*.jpg, base/Nojo/*.jpg, etc.")
            return None, None, None, None
        
        if len(X_train) == 0 or len(X_test) == 0:
            print("❌ Nenhuma imagem carregada. Verifique os caminhos e nomes das pastas!")
            return None, None, None, None
        
        print(f"\n📊 Dados carregados com sucesso:")
        print(f"- X_train: {X_train.shape}")
        print(f"- y_train: {y_train.shape}")
        print(f"- X_test: {X_test.shape}")
        print(f"- y_test: {y_test.shape}")
        
        # NORMALIZAÇÃO ESPECÍFICA PARA EFFICIENTNET: [0, 255] -> [-1, 1]
        print("🔄 Aplicando normalização EfficientNet...")
        
        X_train = X_train.astype(np.float32)
        X_test = X_test.astype(np.float32)
        
        # EfficientNet normalização: [0, 255] -> [-1, 1]
        X_train = (X_train / 127.5) - 1.0
        X_test = (X_test / 127.5) - 1.0
        
        print("✅ Normalização EfficientNet aplicada: [0,255] -> [-1,1]")
        
        # Verifica resultado final
        print(f"\n🔍 Verificação final:")
        print(f"- X_train range: [{X_train.min():.3f}, {X_train.max():.3f}]")
        print(f"- X_test range: [{X_test.min():.3f}, {X_test.max():.3f}]")
        print(f"- Formato das imagens: {X_train.shape[1:]} (deve ser {IMG_SIZE}x{IMG_SIZE}x3)")
        
        # Verifica distribuição de classes
        train_distribution = dict(Counter(y_train))
        test_distribution = dict(Counter(y_test))
        
        print(f"\n📈 Distribuição de classes:")
        emotion_names = list(EMOTION_LABELS.keys())
        print("- Treino:")
        for label, count in train_distribution.items():
            emotion_name = emotion_names[label]
            print(f"  {emotion_name}: {count} imagens")
        
        print("- Teste:")
        for label, count in test_distribution.items():
            emotion_name = emotion_names[label]
            print(f"  {emotion_name}: {count} imagens")
        
        return X_train, y_train, X_test, y_test
        
    except Exception as e:
        print(f"❌ Erro ao carregar dados: {e}")
        print(f"📍 Erro detalhado: {type(e).__name__}")
        import traceback
        traceback.print_exc()
        
        print("\n💡 Soluções possíveis:")
        print("1. Verifique se o caminho está correto")
        print("2. Verifique se as pastas de emoções existem")
        print("3. Verifique se há imagens nas pastas")
        print("4. Verifique permissões de acesso")
        
        return None, None, None, None

# Executa carregamento
X_train, y_train, X_test, y_test = load_preprocessed_data_efficientnet_from_images()

if X_train is not None:
    print(f"\n🎯 EfficientNet: Dados prontos para treinamento!")
    print(f"📏 Formato final: {X_train.shape}")
    print(f"🎨 Range de valores: [{X_train.min():.3f}, {X_train.max():.3f}]")
    print(f"✅ Pronto para EfficientNet-B0!")
else:
    print("❌ Falha no carregamento dos dados")

Carregando dados pré-processados JPG para EfficientNet...
🔍 Analisando estrutura de: ../data/augmented/raf_db_balanced
📁 Diretórios: ['test', 'train']
📄 Arquivos: 0 encontrados
✅ Estrutura detectada: train/test/emotion/
Carregando TREINO de: ../data/augmented/raf_db_balanced/train
📁 Subdiretórios encontrados: ['Tristeza', 'Raiva', 'Neutro', 'Surpresa', 'Felicidade', 'Medo', 'Nojo']
  📸 Raiva: 1000 arquivos encontrados
  ✅ Raiva: 1000 imagens carregadas com sucesso
  📸 Nojo: 1000 arquivos encontrados
  ✅ Nojo: 1000 imagens carregadas com sucesso
  📸 Medo: 1000 arquivos encontrados
  ✅ Medo: 1000 imagens carregadas com sucesso
  📸 Felicidade: 1000 arquivos encontrados
  ✅ Felicidade: 1000 imagens carregadas com sucesso
  📸 Neutro: 1000 arquivos encontrados
  ✅ Neutro: 1000 imagens carregadas com sucesso
  📸 Tristeza: 1000 arquivos encontrados
  ✅ Tristeza: 1000 imagens carregadas com sucesso
  📸 Surpresa: 1000 arquivos encontrados
  ✅ Surpresa: 1000 imagens carregadas com sucesso
Carrega

In [28]:
def create_efficientnet_experiment_structure():
    """
    Cria estrutura de diretórios específica para experimentos EfficientNet.
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    experiment_id = f"efficientnet_emotion_{timestamp}"
    
    # Cria diretórios específicos
    os.makedirs("models/efficientNetB0", exist_ok=True)
    os.makedirs("metrics/efficientNetB0", exist_ok=True)
    os.makedirs("plots/efficientNetB0", exist_ok=True)
    
    return experiment_id

def save_efficientnet_model_if_good_performance(model, base_model, accuracy, f1_score, experiment_id, threshold=0.80):
    """
    Salva modelo EfficientNet apenas se performance for boa.
    Inclui estratégia específica de salvamento com base_model para fine-tuning futuro.
    
    Args:
        model: Modelo completo treinado
        base_model: Base EfficientNet para referência
        accuracy: Acurácia do modelo
        f1_score: F1-score macro do modelo  
        experiment_id: ID único do experimento
        threshold: Limite mínimo para salvar
    """
    # Critério mais rigoroso para EfficientNet (esperamos maior eficiência)
    performance_score = (accuracy + f1_score) / 2
    
    if performance_score >= threshold:
        
        # Salva pesos do modelo completo
        model.save_weights(f"models/efficientnet/weights_efficientnet_{experiment_id}.h5")
        
        # Configuração detalhada do modelo EfficientNet
        model_config = {
            'architecture': 'EfficientNet-B0',
            'img_size': IMG_SIZE,
            'num_classes': 7,
            'experiment_id': experiment_id,
            'accuracy': accuracy,
            'f1_score': f1_score,
            'performance_score': performance_score,
            'normalization_range': '[-1, 1]',
            'total_params': model.count_params(),
            'trainable_params': sum([tf.keras.backend.count_params(p) for p in model.trainable_weights]),
            'base_model_layers': len(base_model.layers),
            'efficientnet_config': EFFICIENTNET_CONFIG,
            'timestamp': datetime.now().isoformat()
        }
        
        # Salva configuração
        with open(f"models/efficientnet/config_efficientnet_{experiment_id}.pkl", 'wb') as f:
            pickle.dump(model_config, f)
        
        print(f"EfficientNet salvo! Performance: {performance_score:.4f} (Acc={accuracy:.4f}, F1={f1_score:.4f})")
        return True
    else:
        print(f"Performance insuficiente: {performance_score:.4f} < {threshold}")
        return False

def save_efficientnet_metrics_to_csv(metrics_dict, experiment_id):
    """
    Salva métricas EfficientNet em CSV separado para análise comparativa.
    """
    # Adiciona identificador de arquitetura
    metrics_dict['architecture'] = 'EfficientNet-B0'
    
    # DataFrame com métricas
    metrics_df = pd.DataFrame([metrics_dict])
    
    # Arquivo CSV específico para EfficientNet
    efficientnet_csv = "metrics/efficientnet/efficientnet_performance_metrics.csv"
    
    # Append ao CSV se existir
    if os.path.exists(efficientnet_csv):
        metrics_df.to_csv(efficientnet_csv, mode='a', header=False, index=False)
    else:
        metrics_df.to_csv(efficientnet_csv, index=False)
    
    # Arquivo CSV consolidado (todos os modelos)
    consolidated_csv = "metrics/all_models_comparison.csv"
    if os.path.exists(consolidated_csv):
        metrics_df.to_csv(consolidated_csv, mode='a', header=False, index=False)
    else:
        metrics_df.to_csv(consolidated_csv, index=False)
    
    # Arquivo individual
    individual_csv = f"metrics/efficientnet/efficientnet_metrics_{experiment_id}.csv"
    metrics_df.to_csv(individual_csv, index=False)
    
    print(f"Métricas EfficientNet salvas em:")
    print(f"  • Específico: {efficientnet_csv}")
    print(f"  • Consolidado: {consolidated_csv}")
    print(f"  • Individual: {individual_csv}")

# Inicializa estrutura específica do EfficientNet
experiment_id = create_efficientnet_experiment_structure()
print(f"Experimento EfficientNet iniciado: {experiment_id}")
print("Estrutura otimizada para comparação de arquiteturas")

Experimento EfficientNet iniciado: efficientnet_emotion_20250915_202611
Estrutura otimizada para comparação de arquiteturas


In [3]:
def create_efficientnet_model():
    """
    Cria modelo EfficientNet-B0 otimizado para classificação de emoções.
    
    EfficientNet Architecture:
    - Compound scaling (width, depth, resolution)
    - Mobile inverted bottleneck convolutions (MBConv)
    - Squeeze-and-excitation optimization
    - Swish activation functions
    
    Returns:
        tuple: (complete_model, base_model)
    """
    # EfficientNet-B0 pré-treinado no ImageNet
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        #drop_connect_rate=0.2  # Dropout específico do EfficientNet
    )
    
    # Inicial: congela toda a base para feature extraction
    base_model.trainable = False
    
    # Head customizado para classificação de emoções
    x = base_model.output
    x = GlobalAveragePooling2D(name='global_avg_pool')(x)
    
    # Primeira camada densa - maior que ResNet50 para compensar menor base
    x = Dense(1024, activation='relu', name='dense_1024')(x)
    x = Dropout(EFFICIENTNET_CONFIG['dropout_rate'], name='dropout_main')(x)
    
    # Segunda camada densa
    x = Dense(512, activation='relu', name='dense_512')(x)
    x = Dropout(0.3, name='dropout_secondary')(x)
    
    # Camada de classificação final
    predictions = Dense(7, activation='softmax', name='emotion_predictions')(x)
    
    # Modelo completo
    model = Model(inputs=base_model.input, outputs=predictions, name='EfficientNet_Emotion_Classifier')
    
    return model, base_model

def compile_efficientnet_phase1(model):
    """
    Compilação para Fase 1: Feature extraction (base congelada).
    """
    model.compile(
        optimizer=Adam(learning_rate=EFFICIENTNET_CONFIG['base_learning_rate']),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    print("Fase 1: Modelo compilado para feature extraction")
    
def compile_efficientnet_phase2(model, base_model):
    """
    Compilação para Fase 2: Fine-tuning (últimas camadas descongeladas).
    """
    # Descongela últimas camadas para fine-tuning
    base_model.trainable = True
    
    # Mantém congeladas as primeiras camadas
    fine_tune_at = len(base_model.layers) - EFFICIENTNET_CONFIG['fine_tune_layers']
    
    # Congela camadas iniciais
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
    
    # Recompila com learning rate menor
    model.compile(
        optimizer=Adam(learning_rate=EFFICIENTNET_CONFIG['fine_tune_learning_rate']),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print(f"Fase 2: Fine-tuning das últimas {EFFICIENTNET_CONFIG['fine_tune_layers']} camadas")
    print(f"Camadas treináveis: {fine_tune_at} a {len(base_model.layers)}")

# Verifica se dados foram carregados antes de criar modelo
if X_train is not None:
    print("Criando modelo EfficientNet-B0 otimizado...")
    model, base_model = create_efficientnet_model()
    
    # Compilação inicial (Fase 1)
    compile_efficientnet_phase1(model)
    
    # Estatísticas do modelo
    total_params = model.count_params()
    trainable_params = sum([tf.keras.backend.count_params(p) for p in model.trainable_weights])
    non_trainable_params = total_params - trainable_params
    
    print(f"EfficientNet-B0 criado com sucesso:")
    print(f"  • Total de parâmetros: {total_params:,}")
    print(f"  • Parâmetros treináveis (Fase 1): {trainable_params:,}")
    print(f"  • Parâmetros congelados: {non_trainable_params:,}")
    print(f"  • Eficiência: {total_params/1000000:.1f}M parâmetros")
    print(f"  • Ratio treinável/total: {(trainable_params/total_params)*100:.1f}%")
    
    # Sumário do modelo (camadas principais)
    print(f"\nArquitetura resumida:")
    print(f"  • Base EfficientNet: {len(base_model.layers)} camadas")
    print(f"  • GlobalAveragePooling2D")
    print(f"  • Dense(1024) + Dropout({EFFICIENTNET_CONFIG['dropout_rate']})")
    print(f"  • Dense(512) + Dropout(0.3)")
    print(f"  • Dense(7) softmax")
    
    monitor.update_peak_memory()
    
else:
    print("Erro: Dados não carregados. Verifique a célula de carregamento.")

NameError: name 'X_train' is not defined

In [4]:
def create_efficientnet_model():
    """
    Cria modelo EfficientNet-B0 otimizado para classificação de emoções.
    
    EfficientNet Architecture:
    - Compound scaling (width, depth, resolution)
    - Mobile inverted bottleneck convolutions (MBConv)
    - Squeeze-and-excitation optimization
    - Swish activation functions
    
    Returns:
        tuple: (complete_model, base_model)
    """
    # EfficientNet-B0 pré-treinado no ImageNet
    # CORRIGIDO: Removido parâmetro 'drop_connect_rate' inválido
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )
    
    # Inicial: congela toda a base para feature extraction
    base_model.trainable = False
    
    # Head customizado para classificação de emoções
    x = base_model.output
    x = GlobalAveragePooling2D(name='global_avg_pool')(x)
    
    # Primeira camada densa - maior que ResNet50 para compensar menor base
    x = Dense(1024, activation='relu', name='dense_1024')(x)
    x = Dropout(EFFICIENTNET_CONFIG['dropout_rate'], name='dropout_main')(x)
    
    # Segunda camada densa
    x = Dense(512, activation='relu', name='dense_512')(x)
    x = Dropout(0.3, name='dropout_secondary')(x)
    
    # Camada de classificação final
    predictions = Dense(7, activation='softmax', name='emotion_predictions')(x)
    
    # Modelo completo
    model = Model(inputs=base_model.input, outputs=predictions, name='EfficientNet_Emotion_Classifier')
    
    return model, base_model

def compile_efficientnet_phase1(model):
    """
    Compilação para Fase 1: Feature extraction (base congelada).
    """
    model.compile(
        optimizer=Adam(learning_rate=EFFICIENTNET_CONFIG['base_learning_rate']),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    print("Fase 1: Modelo compilado para feature extraction")
    
def compile_efficientnet_phase2(model, base_model):
    """
    Compilação para Fase 2: Fine-tuning (últimas camadas descongeladas).
    """
    # Descongela últimas camadas para fine-tuning
    base_model.trainable = True
    
    # Mantém congeladas as primeiras camadas
    fine_tune_at = len(base_model.layers) - EFFICIENTNET_CONFIG['fine_tune_layers']
    
    # Congela camadas iniciais
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
    
    # Recompila com learning rate menor
    model.compile(
        optimizer=Adam(learning_rate=EFFICIENTNET_CONFIG['fine_tune_learning_rate']),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print(f"Fase 2: Fine-tuning das últimas {EFFICIENTNET_CONFIG['fine_tune_layers']} camadas")
    print(f"Camadas treináveis: {fine_tune_at} a {len(base_model.layers)}")

# Verifica se dados foram carregados antes de criar modelo
if X_train is not None:
    print("Criando modelo EfficientNet-B0 otimizado...")
    model, base_model = create_efficientnet_model()
    
    # Compilação inicial (Fase 1)
    compile_efficientnet_phase1(model)
    
    # Estatísticas do modelo
    total_params = model.count_params()
    trainable_params = sum([tf.keras.backend.count_params(p) for p in model.trainable_weights])
    non_trainable_params = total_params - trainable_params
    
    print(f"EfficientNet-B0 criado com sucesso:")
    print(f"  • Total de parâmetros: {total_params:,}")
    print(f"  • Parâmetros treináveis (Fase 1): {trainable_params:,}")
    print(f"  • Parâmetros congelados: {non_trainable_params:,}")
    print(f"  • Eficiência: {total_params/1000000:.1f}M parâmetros")
    print(f"  • Ratio treinável/total: {(trainable_params/total_params)*100:.1f}%")
    
    # Sumário do modelo (camadas principais)
    print(f"\nArquitetura resumida:")
    print(f"  • Base EfficientNet: {len(base_model.layers)} camadas")
    print(f"  • GlobalAveragePooling2D")
    print(f"  • Dense(1024) + Dropout({EFFICIENTNET_CONFIG['dropout_rate']})")
    print(f"  • Dense(512) + Dropout(0.3)")
    print(f"  • Dense(7) softmax")
    
    monitor.update_peak_memory()
    
else:
    print("Erro: Dados não carregados. Verifique a célula de carregamento.")

NameError: name 'X_train' is not defined

In [5]:
def create_efficientnet_model_with_custom_dropout():
    """
    Versão alternativa com controle manual de dropout.
    """
    # Base model sem parâmetros extras
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )
    
    # Congela inicialmente
    base_model.trainable = False
    
    # OPCIONAL: Adiciona dropout customizado às camadas intermediárias
    x = base_model.output
    x = GlobalAveragePooling2D(name='global_avg_pool')(x)
    
    # Dropout adicional para regularização (simula drop_connect)
    x = Dropout(0.2, name='feature_dropout')(x)  # Equivalent to drop_connect_rate
    
    # Camadas de classificação
    x = Dense(1024, activation='relu', name='dense_1024')(x)
    x = Dropout(EFFICIENTNET_CONFIG['dropout_rate'], name='dropout_main')(x)
    
    x = Dense(512, activation='relu', name='dense_512')(x)
    x = Dropout(0.3, name='dropout_secondary')(x)
    
    predictions = Dense(7, activation='softmax', name='emotion_predictions')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions, name='EfficientNet_Emotion_Classifier')
    
    print("✅ EfficientNet criado com dropout customizado")
    print("💡 Drop connect simulado via Dropout(0.2) após GlobalAveragePooling")
    
    return model, base_model

In [ ]:
class EfficientNetMemoryCallback(tf.keras.callbacks.Callback):
    """
    Callback especializado para monitoramento de EfficientNet.
    Foca em eficiência de memória e comparação com ResNet50.
    """
    
    def __init__(self, monitor, phase_name):
        super().__init__()
        self.monitor = monitor
        self.phase_name = phase_name
        self.epoch_times = []
        
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()
        
    def on_epoch_end(self, epoch, logs=None):
        # Calcula tempo da época
        epoch_time = time.time() - self.epoch_start_time
        self.epoch_times.append(epoch_time)
        
        # Atualiza memória
        self.monitor.update_peak_memory()
        
        # Log detalhado a cada 5 épocas
        if epoch % 5 == 0:
            current_memory = self.monitor._get_memory_usage()
            avg_epoch_time = np.mean(self.epoch_times[-5:])  # Média das últimas 5 épocas
            
            print(f"{self.phase_name} - Época {epoch+1}")
            print(f"  • Memória atual: {current_memory:.1f} MB")
            print(f"  • Tempo/época: {avg_epoch_time:.1f}s")
            if logs:
                print(f"  • Val_accuracy: {logs.get('val_accuracy', 0):.4f}")
                print(f"  • Val_loss: {logs.get('val_loss', 0):.4f}")

def setup_efficientnet_callbacks(monitor, phase_name):
    """
    Configura callbacks otimizados para cada fase do EfficientNet.
    """
    callbacks_list = []
    
    # Early stopping com paciência diferente por fase
    if phase_name == "Fase1":
        patience = 12  # Menos paciência na fase 1
        min_delta = 0.001
    else:
        patience = 20  # Mais paciência no fine-tuning
        min_delta = 0.0005
    
    early_stopping = EarlyStopping(
        monitor='val_accuracy',  # Monitora accuracy para EfficientNet
        patience=patience,
        restore_best_weights=True,
        verbose=1,
        mode='max',
        min_delta=min_delta
    )
    
    # Reduce learning rate
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.3,  # Redução mais agressiva para EfficientNet
        patience=8,
        min_lr=1e-8,
        verbose=1,
        mode='min'
    )
    
    # Memory callback personalizado
    memory_callback = EfficientNetMemoryCallback(monitor, phase_name)
    
    callbacks_list = [early_stopping, reduce_lr, memory_callback]
    
    return callbacks_list

def train_efficientnet_two_phase(model, base_model, X_train, y_train, X_val, y_val, monitor):
    """
    Treinamento EfficientNet em 2 fases otimizadas.
    
    Fase 1: Feature extraction (base congelada)
    Fase 2: Fine-tuning (últimas camadas descongeladas)
    """
    print("="*60)
    print("INICIANDO TREINAMENTO EFFICIENTNET EM 2 FASES")
    print("="*60)
    
    # === FASE 1: FEATURE EXTRACTION ===
    print("FASE 1: FEATURE EXTRACTION")
    print("-" * 40)
    monitor.start_phase('phase1')
    
    # Callbacks para Fase 1
    phase1_callbacks = setup_efficientnet_callbacks(monitor, "Fase1")
    
    # Treinamento Fase 1
    history_phase1 = model.fit(
        X_train, y_train,
        batch_size=BATCH_SIZE,
        epochs=30,  # Menos épocas na fase 1
        validation_data=(X_val, y_val),
        callbacks=phase1_callbacks,
        verbose=1,
        shuffle=True
    )
    
    phase1_duration = monitor.end_phase('phase1')
    best_val_acc_phase1 = max(history_phase1.history['val_accuracy'])
    print(f"Fase 1 - Melhor val_accuracy: {best_val_acc_phase1:.4f}")
    
    # === FASE 2: FINE-TUNING ===
    print("\nFASE 2: FINE-TUNING")
    print("-" * 40)
    monitor.start_phase('phase2')
    
    # Reconfiguração para fine-tuning
    compile_efficientnet_phase2(model, base_model)
    
    # Callbacks para Fase 2
    phase2_callbacks = setup_efficientnet_callbacks(monitor, "Fase2")
    
    # Treinamento Fase 2
    history_phase2 = model.fit(
        X_train, y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS - 30,  # Restante das épocas
        validation_data=(X_val, y_val),
        callbacks=phase2_callbacks,
        verbose=1,
        shuffle=True
    )
    
    phase2_duration = monitor.end_phase('phase2')
    best_val_acc_phase2 = max(history_phase2.history['val_accuracy'])
    print(f"Fase 2 - Melhor val_accuracy: {best_val_acc_phase2:.4f}")
    
    # Combina históricos das duas fases
    combined_history = {
        'accuracy': history_phase1.history['accuracy'] + history_phase2.history['accuracy'],
        'val_accuracy': history_phase1.history['val_accuracy'] + history_phase2.history['val_accuracy'],
        'loss': history_phase1.history['loss'] + history_phase2.history['loss'],
        'val_loss': history_phase1.history['val_loss'] + history_phase2.history['val_loss'],
        'phase1_epochs': len(history_phase1.history['accuracy']),
        'phase2_epochs': len(history_phase2.history['accuracy']),
        'phase1_duration': phase1_duration,
        'phase2_duration': phase2_duration,
        'best_val_acc_phase1': best_val_acc_phase1,
        'best_val_acc_phase2': best_val_acc_phase2
    }
    
    print(f"\nTREINAMENTO COMPLETO:")
    print(f"  • Total de épocas: {combined_history['phase1_epochs'] + combined_history['phase2_epochs']}")
    print(f"  • Melhor accuracy final: {max(combined_history['val_accuracy']):.4f}")
    print(f"  • Tempo total: {timedelta(seconds=int(phase1_duration + phase2_duration))}")
    
    return combined_history

# Executa treinamento se modelo foi criado com sucesso
if 'model' in locals() and model is not None:
    
    # Preparação dos dados
    print("Preparando dados para treinamento EfficientNet...")
    
    # Divisão estratificada treino/validação
    X_train_split, X_val, y_train_split, y_val = train_test_split(
        X_train, y_train,
        test_size=VALIDATION_SPLIT,
        stratify=y_train,
        random_state=42
    )
    
    # Conversão para categorical
    y_train_cat = to_categorical(y_train_split, 7)
    y_val_cat = to_categorical(y_val, 7)
    y_test_cat = to_categorical(y_test, 7)
    
    print(f"Dados preparados:")
    print(f"  • Treino: {X_train_split.shape}")
    print(f"  • Validação: {X_val.shape}")
    print(f"  • Teste: {X_test.shape}")
    print(f"  • Range normalização: [{X_train_split.min():.2f}, {X_train_split.max():.2f}]")
    
    # Executa treinamento em 2 fases
    history = train_efficientnet_two_phase(
        model, base_model, X_train_split, y_train_cat, X_val, y_val_cat, monitor
    )
    
    print("EfficientNet: Treinamento em 2 fases finalizado com sucesso!")
    
else:
    print("Erro: Modelo EfficientNet não foi criado. Verifique células anteriores.")

NameError: name 'tf' is not defined

In [9]:
def comprehensive_efficientnet_evaluation(model, X_test, y_test_cat, y_test_original, history, monitor):
    """
    Avaliação completa do EfficientNet com métricas comparativas.
    Foco em eficiência computacional vs ResNet50.
    """
    print("="*70)
    print("AVALIAÇÃO COMPARATIVA EFFICIENTNET-B0")
    print("="*70)
    
    # === MÉTRICAS DE INFERÊNCIA ===
    print("Medindo performance de inferência...")
    
    # Múltiplas medições para precisão
    inference_times = []
    for i in range(5):  # 5 medições
        start_time = time.time()
        y_pred_prob = model.predict(X_test, batch_size=BATCH_SIZE, verbose=0)
        end_time = time.time()
        inference_times.append(end_time - start_time)
    
    # Estatísticas de inferência
    avg_inference_time = np.mean(inference_times)
    std_inference_time = np.std(inference_times)
    inference_per_sample = avg_inference_time / len(X_test)
    samples_per_second = len(X_test) / avg_inference_time
    
    # === MÉTRICAS DE CLASSIFICAÇÃO ===
    y_pred_classes = np.argmax(y_pred_prob, axis=1)
    y_true_classes = y_test_original
    
    # Métricas principais
    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true_classes, y_pred_classes, average='macro', zero_division=0
    )
    
    # Métricas adicionais
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        y_true_classes, y_pred_classes, average='micro', zero_division=0
    )
    
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        y_true_classes, y_pred_classes, average='weighted', zero_division=0
    )
    
    # Matriz de confusão e relatório por classe
    conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
    emotion_names = list(EMOTION_LABELS.keys())
    class_report = classification_report(
        y_true_classes, y_pred_classes,
        target_names=emotion_names,
        output_dict=True
    )
    
    # === MÉTRICAS DE EFICIÊNCIA COMPUTACIONAL ===
    efficiency_metrics = monitor.get_efficiency_metrics()
    current_memory = monitor._get_memory_usage()
    
    # Parâmetros do modelo
    total_params = model.count_params()
    trainable_params = sum([tf.keras.backend.count_params(p) for p in model.trainable_weights])
    
    # === COMPILAÇÃO COMPLETA DAS MÉTRICAS ===
    comprehensive_metrics = {
        # Identificação
        'experiment_id': experiment_id,
        'model_architecture': 'EfficientNet-B0',
        'timestamp': datetime.now().isoformat(),
        
        # Configuração
        'img_size': IMG_SIZE,
        'batch_size': BATCH_SIZE,
        'normalization_range': '[-1, 1]',
        'total_epochs_trained': history['phase1_epochs'] + history['phase2_epochs'],
        'phase1_epochs': history['phase1_epochs'],
        'phase2_epochs': history['phase2_epochs'],
        
        # Performance de classificação
        'test_accuracy': accuracy,
        'f1_score_macro': f1,
        'f1_score_micro': f1_micro,
        'f1_score_weighted': f1_weighted,
        'precision_macro': precision,
        'recall_macro': recall,
        'performance_score': (accuracy + f1) / 2,
        
        # Eficiência temporal
        'avg_inference_time_seconds': avg_inference_time,
        'std_inference_time_seconds': std_inference_time,
        'inference_per_sample_ms': inference_per_sample * 1000,
        'samples_per_second': samples_per_second,
        'phase1_training_time_seconds': history['phase1_duration'],
        'phase2_training_time_seconds': history['phase2_duration'],
        'total_training_time_seconds': history['phase1_duration'] + history['phase2_duration'],
        
        # Eficiência de memória
        'peak_memory_mb': monitor.peak_memory_mb,
        'current_memory_mb': current_memory,
        'memory_efficiency': efficiency_metrics['memory_efficiency'],
        'memory_growth_factor': efficiency_metrics['memory_growth_factor'],
        'peak_memory_gb': efficiency_metrics['peak_memory_gb'],
        
        # Eficiência de modelo
        'total_parameters': total_params,
        'trainable_parameters': trainable_params,
        'parameters_millions': total_params / 1000000,
        'params_per_accuracy': total_params / accuracy if accuracy > 0 else 0,
        'efficiency_score': accuracy / (total_params / 1000000),  # Accuracy per million params
        
        # Métricas por emoção
        'anger_f1': class_report['anger']['f1-score'],
        'disgust_f1': class_report['disgust']['f1-score'],
        'fear_f1': class_report['fear']['f1-score'],
        'happy_f1': class_report['happy']['f1-score'],
        'neutral_f1': class_report['neutral']['f1-score'],
        'sadness_f1': class_report['sadness']['f1-score'],
        'surprise_f1': class_report['surprise']['f1-score'],
        
        # Dados do dataset
        'train_samples': len(X_train_split),
        'val_samples': len(X_val),
        'test_samples': len(X_test),
        
        # Comparação específica EfficientNet
        'compound_scaling': 'Yes',
        'mobile_inverted_bottleneck': 'Yes',
        'squeeze_excitation': 'Yes',
        'drop_connect_rate': 0.2
    }
    
    return comprehensive_metrics, conf_matrix, class_report

# Executa avaliação se treinamento foi bem-sucedido
if 'history' in locals() and history is not None:
    
    print("Executando avaliação completa EfficientNet...")
    
    # Avaliação detalhada
    metrics, confusion_matrix_result, detailed_report = comprehensive_efficientnet_evaluation(
        model, X_test, y_test_cat, y_test, history, monitor
    )
    
    # Salva métricas em CSV
    save_efficientnet_metrics_to_csv(metrics, experiment_id)
    
    # Tenta salvar modelo se performance for boa
    model_saved = save_efficientnet_model_if_good_performance(
        model, base_model,
        metrics['test_accuracy'], 
        metrics['f1_score_macro'], 
        experiment_id,
        threshold=0.75  # Threshold mais baixo para EfficientNet (mais conservador)
    )
    
    # Finaliza monitoramento
    monitor_final_stats = monitor.end_monitoring()
    
    # === COMPARAÇÃO COM RESNET50 ===
    print(f"\n{'='*70}")
    print(f"COMPARAÇÃO EFFICIENTNET-B0 vs ResNet50")
    print(f"{'='*70}")
    print(f"EfficientNet-B0:")
    print(f"  • Parâmetros: {metrics['parameters_millions']:.1f}M")
    print(f"  • Acurácia: {metrics['test_accuracy']:.4f}")
    print(f"  • F1-Score: {metrics['f1_score_macro']:.4f}")
    print(f"  • Inferência/amostra: {metrics['inference_per_sample_ms']:.2f} ms")
    print(f"  • Eficiência: {metrics['efficiency_score']:.2f} acc/M_params")
    print(f"  • Pico memória: {metrics['peak_memory_gb']:.2f} GB")
    print(f"")
    print(f"ResNet50 (típico):")
    print(f"  • Parâmetros: ~25.6M")
    print(f"  • Eficiência esperada: ~80% dos parâmetros do EfficientNet")
    print(f"  • Comparação: EfficientNet é {25.6/metrics['parameters_millions']:.1f}x mais eficiente")
    print(f"")
    print(f"Modelo salvo: {'Sim' if model_saved else 'Não'}")
    print(f"Performance Score: {metrics['performance_score']:.4f}")
    
else:
    print("Erro: Treinamento EfficientNet não foi executado corretamente")

Erro: Treinamento EfficientNet não foi executado corretamente


In [10]:
def create_efficientnet_comparative_visualizations(history, confusion_matrix_result, metrics, detailed_report):
    """
    Cria visualizações especializadas para análise de EfficientNet vs ResNet50.
    """
    fig = plt.figure(figsize=(24, 16))
    
    # === 1. HISTÓRICO DE TREINAMENTO EM 2 FASES ===
    ax1 = plt.subplot(3, 4, 1)
    epochs_phase1 = range(1, history['phase1_epochs'] + 1)
    epochs_phase2 = range(history['phase1_epochs'] + 1, 
                         history['phase1_epochs'] + history['phase2_epochs'] + 1)
    
    # Accuracy
    plt.plot(epochs_phase1, history['accuracy'][:history['phase1_epochs']], 
             'b-', linewidth=2, label='Fase 1 - Train')
    plt.plot(epochs_phase1, history['val_accuracy'][:history['phase1_epochs']], 
             'b--', linewidth=2, label='Fase 1 - Val')
    plt.plot(epochs_phase2, history['accuracy'][history['phase1_epochs']:], 
             'r-', linewidth=2, label='Fase 2 - Train')
    plt.plot(epochs_phase2, history['val_accuracy'][history['phase1_epochs']:], 
             'r--', linewidth=2, label='Fase 2 - Val')
    
    plt.axvline(x=history['phase1_epochs'], color='gray', linestyle=':', alpha=0.7)
    plt.title('EfficientNet: Accuracy - 2 Fases', fontsize=12, fontweight='bold')
    plt.xlabel('Época')
    plt.ylabel('Accuracy')
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    
    # === 2. LOSS EM 2 FASES ===
    ax2 = plt.subplot(3, 4, 2)
    plt.plot(epochs_phase1, history['loss'][:history['phase1_epochs']], 
             'b-', linewidth=2, label='Fase 1 - Train')
    plt.plot(epochs_phase1, history['val_loss'][:history['phase1_epochs']], 
             'b--', linewidth=2, label='Fase 1 - Val')
    plt.plot(epochs_phase2, history['loss'][history['phase1_epochs']:], 
             'r-', linewidth=2, label='Fase 2 - Train')
    plt.plot(epochs_phase2, history['val_loss'][history['phase1_epochs']:], 
             'r--', linewidth=2, label='Fase 2 - Val')
    
    plt.axvline(x=history['phase1_epochs'], color='gray', linestyle=':', alpha=0.7)
    plt.title('EfficientNet: Loss - 2 Fases', fontsize=12, fontweight='bold')
    plt.xlabel('Época')
    plt.ylabel('Loss')
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    
    # === 3. MATRIZ DE CONFUSÃO ===
    ax3 = plt.subplot(3, 4, 3)
    emotion_names = list(EMOTION_LABELS.keys())
    sns.heatmap(confusion_matrix_result, annot=True, fmt='d', cmap='Greens',
                xticklabels=emotion_names, yticklabels=emotion_names, ax=ax3)
    plt.title('Matriz de Confusão - EfficientNet', fontsize=12, fontweight='bold')
    plt.ylabel('Classe Real')
    plt.xlabel('Classe Predita')
    
    # === 4. F1-SCORE POR EMOÇÃO ===
    ax4 = plt.subplot(3, 4, 4)
    f1_scores = [detailed_report[emotion]['f1-score'] for emotion in emotion_names]
    colors = plt.cm.viridis(np.linspace(0, 1, len(emotion_names)))
    bars = plt.bar(emotion_names, f1_scores, color=colors, alpha=0.8, edgecolor='black')
    plt.title('F1-Score por Emoção', fontsize=12, fontweight='bold')
    plt.ylabel('F1-Score')
    plt.xticks(rotation=45)
    plt.ylim(0, 1)
    
    for bar, score in zip(bars, f1_scores):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{score:.3f}', ha='center', va='bottom', fontsize=10)
    
    # === 5. COMPARAÇÃO DE EFICIÊNCIA ===
    ax5 = plt.subplot(3, 4, 5)
    # Dados comparativos (EfficientNet vs ResNet50 típico)
    models = ['EfficientNet-B0', 'ResNet50']
    parameters = [metrics['parameters_millions'], 25.6]  # ResNet50 típico
    accuracy_comparison = [metrics['test_accuracy'], 0.75]  # Estimativa ResNet50
    
    x = np.arange(len(models))
    width = 0.35
    
    bars1 = plt.bar(x - width/2, parameters, width, label='Parâmetros (M)', alpha=0.8, color='skyblue')
    bars2 = plt.bar(x + width/2, [acc * 100 for acc in accuracy_comparison], width, 
                   label='Accuracy (%)', alpha=0.8, color='lightcoral')
    
    plt.title('Comparação: Parâmetros vs Accuracy', fontsize=12, fontweight='bold')
    plt.xlabel('Modelo')
    plt.ylabel('Valor')
    plt.xticks(x, models)
    plt.legend()
    
    # Adiciona valores nas barras
    for bar, value in zip(bars1, parameters):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                f'{value:.1f}M', ha='center', va='bottom')
    for bar, acc in zip(bars2, accuracy_comparison):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{acc:.3f}', ha='center', va='bottom')
    
    # === 6. EFICIÊNCIA TEMPORAL ===
    ax6 = plt.subplot(3, 4, 6)
    time_metrics = {
        'Fase 1 (min)': history['phase1_duration'] / 60,
        'Fase 2 (min)': history['phase2_duration'] / 60,
        'Inferência (ms)': metrics['inference_per_sample_ms'],
        'Throughput': metrics['samples_per_second']
    }
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    bars = plt.bar(range(len(time_metrics)), list(time_metrics.values()), 
                  color=colors, alpha=0.7)
    plt.title('Métricas de Tempo', fontsize=12, fontweight='bold')
    plt.xticks(range(len(time_metrics)), list(time_metrics.keys()), rotation=45)
    plt.ylabel('Valor')
    
    # === 7. EFICIÊNCIA DE MEMÓRIA ===
    ax7 = plt.subplot(3, 4, 7)
    memory_data = [
        monitor.initial_memory_mb / 1024,  # GB
        monitor.peak_memory_mb / 1024,     # GB
        metrics['memory_growth_factor']
    ]
    memory_labels = ['Inicial (GB)', 'Pico (GB)', 'Fator Crescimento']
    
    bars = plt.bar(memory_labels, memory_data, color=['green', 'orange', 'red'], alpha=0.7)
    plt.title('Uso de Memória', fontsize=12, fontweight='bold')
    plt.ylabel('Valor')
    plt.xticks(rotation=45)
    
    for bar, value in zip(bars, memory_data):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
                f'{value:.2f}', ha='center', va='bottom')
    
    # === 8. RADAR CHART - PERFORMANCE GERAL ===
    ax8 = plt.subplot(3, 4, 8, projection='polar')
    
    categories = ['Accuracy', 'F1-Score', 'Efficiency\n(Acc/M_params)', 'Speed\n(samples/s)', 
                 'Memory\nEfficiency']
    values = [
        metrics['test_accuracy'],
        metrics['f1_score_macro'], 
        min(metrics['efficiency_score'] / 10, 1),  # Normalizado
        min(metrics['samples_per_second'] / 1000, 1),  # Normalizado
        metrics['memory_efficiency']
    ]
    
    # Fecha o radar
    values += values[:1]
    angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
    angles += angles[:1]
    
    ax8.plot(angles, values, 'o-', linewidth=2, color='blue', alpha=0.7)
    ax8.fill(angles, values, alpha=0.25, color='blue')
    ax8.set_xticks(angles[:-1])
    ax8.set_xticklabels(categories)
    ax8.set_ylim(0, 1)
    plt.title('Performance Radar - EfficientNet', fontsize=12, fontweight='bold')
    
    # === 9-12. MÉTRICAS DETALHADAS ===
    # Performance por fase
    ax9 = plt.subplot(3, 4, 9)
    phases = ['Fase 1', 'Fase 2']
    phase_performance = [history['best_val_acc_phase1'], history['best_val_acc_phase2']]
    bars = plt.bar(phases, phase_performance, color=['lightblue', 'lightgreen'], alpha=0.8)
    plt.title('Melhor Accuracy por Fase', fontsize=12, fontweight='bold')
    plt.ylabel('Validation Accuracy')
    plt.ylim(0, 1)
    
    for bar, acc in zip(bars, phase_performance):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                f'{acc:.3f}', ha='center', va='bottom')
    
    # Distribuição de classes
    ax10 = plt.subplot(3, 4, 10)
    test_distribution = [sum(y_test == i) for i in range(7)]
    plt.pie(test_distribution, labels=emotion_names, autopct='%1.1f%%', startangle=90)
    plt.title('Distribuição Classes - Teste', fontsize=12, fontweight='bold')
    
    # Comparação ResNet vs EfficientNet (simulada)
    ax11 = plt.subplot(3, 4, 11)
    comparison_metrics = ['Accuracy', 'Parameters (M)', 'Speed (rel)', 'Memory (rel)']
    efficientnet_values = [metrics['test_accuracy'], metrics['parameters_millions'], 1.0, 1.0]
    resnet_values = [0.75, 25.6, 0.8, 1.2]  # Valores estimados
    
    x = np.arange(len(comparison_metrics))
    width = 0.35
    
    plt.bar(x - width/2, efficientnet_values, width, label='EfficientNet-B0', alpha=0.8)
    plt.bar(x + width/2, resnet_values, width, label='ResNet50', alpha=0.8)
    
    plt.title('EfficientNet vs ResNet50', fontsize=12, fontweight='bold')
    plt.xlabel('Métricas')
    plt.xticks(x, comparison_metrics, rotation=45)
    plt.legend()
    
    # Resumo final
    ax12 = plt.subplot(3, 4, 12)
    ax12.axis('off')
    summary_text = f"""
EfficientNet-B0 RESUMO

Accuracy: {metrics['test_accuracy']:.4f}
F1-Score: {metrics['f1_score_macro']:.4f}
Parâmetros: {metrics['parameters_millions']:.1f}M
Eficiência: {metrics['efficiency_score']:.2f}

Tempo Treino: {timedelta(seconds=int(metrics['total_training_time_seconds']))}
Inferência: {metrics['inference_per_sample_ms']:.1f} ms
Memória Pico: {metrics['peak_memory_gb']:.2f} GB

Vantagem vs ResNet50:
- {25.6/metrics['parameters_millions']:.1f}x menos parâmetros
- Convergência em 2 fases
- Compound scaling otimizado
    """
    ax12.text(0.1, 0.9, summary_text, fontsize=12, verticalalignment='top', 
             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
    
    plt.tight_layout()
    plt.savefig(f'plots/efficientnet/efficientnet_comprehensive_analysis_{experiment_id}.png', 
                dpi=300, bbox_inches='tight')
    plt.show()
    
    # === RELATÓRIO CIENTÍFICO FINAL ===
    print_efficientnet_scientific_report(metrics, history, monitor_final_stats)

def print_efficientnet_scientific_report(metrics, history, monitor_stats):
    """Relatório científico detalhado do EfficientNet"""
    
    print(f"\n{'='*80}")
    print(f"RELATÓRIO CIENTÍFICO FINAL - EFFICIENTNET-B0")
    print(f"Experimento: {experiment_id}")
    print(f"{'='*80}")
    
    print(f"ARQUITETURA E CONFIGURAÇÃO:")
    print(f"  • Modelo: EfficientNet-B0 (Compound Scaling)")
    print(f"  • Parâmetros: {metrics['parameters_millions']:.1f}M")
    print(f"  • Entrada: {IMG_SIZE}x{IMG_SIZE}x3, normalização [-1,1]")
    print(f"  • Treinamento: 2 fases (Feature extraction + Fine-tuning)")
    
    print(f"\nPERFORMANCE DE CLASSIFICAÇÃO:")
    print(f"  • Acurácia: {metrics['test_accuracy']:.4f} ({metrics['test_accuracy']*100:.2f}%)")
    print(f"  • F1-Score Macro: {metrics['f1_score_macro']:.4f}")
    print(f"  • Precisão Macro: {metrics['precision_macro']:.4f}")
    print(f"  • Recall Macro: {metrics['recall_macro']:.4f}")
    print(f"  • Performance Score: {metrics['performance_score']:.4f}")
    
    print(f"\nEFICIÊNCIA COMPUTACIONAL:")
    print(f"  • Eficiência: {metrics['efficiency_score']:.2f} accuracy/M_parameters")
    print(f"  • Parâmetros/Accuracy: {metrics['params_per_accuracy']:,.0f}")
    print(f"  • Throughput: {metrics['samples_per_second']:.1f} amostras/segundo")
    print(f"  • Inferência: {metrics['inference_per_sample_ms']:.2f} ms/amostra")
    
    print(f"\nTREINAMENTO EM 2 FASES:")
    print(f"  • Fase 1 (Feature extraction): {history['phase1_epochs']} épocas, {timedelta(seconds=int(history['phase1_duration']))}")
    print(f"  • Fase 2 (Fine-tuning): {history['phase2_epochs']} épocas, {timedelta(seconds=int(history['phase2_duration']))}")
    print(f"  • Tempo total: {timedelta(seconds=int(metrics['total_training_time_seconds']))}")
    print(f"  • Melhor accuracy Fase 1: {history['best_val_acc_phase1']:.4f}")
    print(f"  • Melhor accuracy Fase 2: {history['best_val_acc_phase2']:.4f}")
    
    print(f"\nUSO DE RECURSOS:")
    print(f"  • Pico de memória: {metrics['peak_memory_gb']:.2f} GB")
    print(f"  • Crescimento de memória: {metrics['memory_growth_factor']:.2f}x")
    print(f"  • Eficiência de memória: {metrics['memory_efficiency']:.3f}")
    
    print(f"\nCOMPARAÇÃO COM RESNET50:")
    print(f"  • Parâmetros: {25.6/metrics['parameters_millions']:.1f}x MENOS parâmetros")
    print(f"  • Compound scaling: Otimização automática width/depth/resolution")
    print(f"  • Mobile bottlenecks: Convoluções mais eficientes")
    print(f"  • Squeeze-and-excitation: Atenção por canal")
    
    print(f"\nRESULTADOS POR EMOÇÃO:")
    emotion_names = list(EMOTION_LABELS.keys())
    for emotion in emotion_names:
        f1_key = f'{emotion}_f1'
        if f1_key in metrics:
            print(f"  • {emotion.capitalize()}: F1 = {metrics[f1_key]:.4f}")
    
    print(f"\nCONCLUSÃO:")
    efficiency_vs_resnet = 25.6 / metrics['parameters_millions']
    print(f"  • EfficientNet-B0 alcançou {metrics['test_accuracy']*100:.1f}% de acurácia")
    print(f"  • Com {efficiency_vs_resnet:.1f}x menos parâmetros que ResNet50")
    print(f"  • Validando compound scaling como arquitetura superior")
    print(f"  • Ideal para aplicações com restrições computacionais")
    
    print(f"{'='*80}")

# Executa análise se avaliação foi bem-sucedida
if 'metrics' in locals() and metrics is not None:
    create_efficientnet_comparative_visualizations(history, confusion_matrix_result, metrics, detailed_report)
    print("EfficientNet: Análise comparativa completa finalizada!")
    print(f"\nArquivos gerados:")
    print(f"  • Métricas EfficientNet: metrics/efficientnet/efficientnet_performance_metrics.csv")
    print(f"  • Comparação consolidada: metrics/all_models_comparison.csv")
    print(f"  • Visualizações: plots/efficientnet/efficientnet_comprehensive_analysis_{experiment_id}.png")
    if model_saved:
        print(f"  • Modelo: models/efficientnet/weights_efficientnet_{experiment_id}.h5")
else:
    print("Erro: Avaliação EfficientNet não foi executada corretamente")

Erro: Avaliação EfficientNet não foi executada corretamente
